In [7]:
from qiskit_optimization.applications import Knapsack
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils import algorithm_globals
from qiskit.algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA

from qiskit_optimization.runtime import QAOAClient

from qiskit.primitives import Sampler

In [17]:
installation_costs = {
    'Supermarket_1': 30000,
    'Church_1': 10000, 
    'Sport_center': 44000
}

traffic_values = {
    'Supermarket_1': [0.06, 0.21, 0.86, 0.38, 0.16, 0.06, 0.38],
    'Church_1': [0.74, 0.74, 0.6, 0.45, 0.22, 0.08, 0.15],
    'Sport_center': [0.30, 0.5, 0.51, 0.50, 0.46, 0.06, 0.73]
}

budget = 60000

In [18]:
traffic_values_list = [sum(l)/len(l) for l in traffic_values.values()]
# demand_from_people = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

# values = list(map(lambda x: (x[0] + x[1]) / 2, list(zip(traffic_values_list, demand_from_people))))
installation_costs_list = list(installation_costs.values())


# Normalize
budget = int((budget / max(installation_costs_list))*100)
installation_costs_list = [int((x / max(installation_costs_list))*100) for x in installation_costs_list]
installation_costs_list

[68, 22, 100]

In [19]:

prob = Knapsack(values=traffic_values_list, weights=installation_costs_list, max_weight=budget)
qp = prob.to_quadratic_program()
print(qp.prettyprint())

Problem name: Knapsack

Maximize
  0.30142857142857143*x_0 + 0.42571428571428577*x_1 + 0.43714285714285717*x_2

Subject to
  Linear constraints (1)
    68*x_0 + 22*x_1 + 100*x_2 <= 136  'c0'

  Binary variables (3)
    x_0 x_1 x_2



In [20]:
# QAOA
meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, sampler=Sampler(), optimizer=COBYLA()))
result = meo.solve(qp)
print(result.prettyprint())
print("\nsolution:", prob.interpret(result))
print("\ntime:", result.min_eigen_solver_result.optimizer_time)

objective function value: 0.862857142857143
variable values: x_0=0.0, x_1=1.0, x_2=1.0
status: SUCCESS

solution: [1, 2]

time: 12.408754110336304


In [ ]:
from qiskit_optimization.converters import QuadraticProgramToQubo

# intermediate QUBO form of the optimization problem
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)
print(qubo.prettyprint())

Problem name: Knapsack

Minimize
  7.827071428571428*c0@int_slack@0^2
  + 31.308285714285713*c0@int_slack@0*c0@int_slack@1
  + 62.616571428571426*c0@int_slack@0*c0@int_slack@2
  + 125.23314285714285*c0@int_slack@0*c0@int_slack@3
  + 250.4662857142857*c0@int_slack@0*c0@int_slack@4
  + 500.9325714285714*c0@int_slack@0*c0@int_slack@5
  + 579.2032857142857*c0@int_slack@0*c0@int_slack@6
  + 31.308285714285713*c0@int_slack@1^2
  + 125.23314285714285*c0@int_slack@1*c0@int_slack@2
  + 250.4662857142857*c0@int_slack@1*c0@int_slack@3
  + 500.9325714285714*c0@int_slack@1*c0@int_slack@4
  + 1001.8651428571428*c0@int_slack@1*c0@int_slack@5
  + 1158.4065714285714*c0@int_slack@1*c0@int_slack@6
  + 125.23314285714285*c0@int_slack@2^2
  + 500.9325714285714*c0@int_slack@2*c0@int_slack@3
  + 1001.8651428571428*c0@int_slack@2*c0@int_slack@4
  + 2003.7302857142856*c0@int_slack@2*c0@int_slack@5
  + 2316.813142857143*c0@int_slack@2*c0@int_slack@6
  + 500.9325714285714*c0@int_slack@3^2
  + 2003.7302857142856*

In [ ]:
# qubit Hamiltonian and offset
op, offset = qubo.to_ising()
print(f"num qubits: {op.num_qubits}, offset: {offset}\n")
print(op)

num qubits: 22, offset: 696933.0405714279

-64515.45571428572 * IIIIIIIIIIIIIIIIIIIIIZ
- 21504.98928571429 * IIIIIIIIIIIIIIIIIIIIZI
- 96773.19107142859 * IIIIIIIIIIIIIIIIIIIZII
- 88171.01878571429 * IIIIIIIIIIIIIIIIIIZIII
- 58063.9257857143 * IIIIIIIIIIIIIIIIIZIIII
- 77418.57985714287 * IIIIIIIIIIIIIIIIZIIIII
- 113977.24421428573 * IIIIIIIIIIIIIIIZIIIIII
- 144084.58192857145 * IIIIIIIIIIIIIIZIIIIIII
- 47311.21828571429 * IIIIIIIIIIIIIZIIIIIIII
- 38709.118857142865 * IIIIIIIIIIIIZIIIIIIIII
- 92472.1842142857 * IIIIIIIIIIIZIIIIIIIIII
- 118278.36821428573 * IIIIIIIIIIZIIIIIIIIIII
- 215051.85092857148 * IIIIIIIIIZIIIIIIIIIIII
- 159138.25135714287 * IIIIIIIIZIIIIIIIIIIIII
- 120428.97950000002 * IIIIIIIZIIIIIIIIIIIIII
- 2150.520214285715 * IIIIIIZIIIIIIIIIIIIIII
- 4301.04042857143 * IIIIIZIIIIIIIIIIIIIIII
- 8602.08085714286 * IIIIZIIIIIIIIIIIIIIIII
- 17204.16171428572 * IIIZIIIIIIIIIIIIIIIIII
- 34408.32342857144 * IIZIIIIIIIIIIIIIIIIIII
- 68816.64685714288 * IZIIIIIIIIIIIIIIIIIIII
- 79569.24